**AI 노바투스과정**

Week 3: 딥러닝 기초 I (2023-01-27)

실습 #3: 신경망 모형의 초모수 결정하기/ Early-Stopping 실습하기

## 1. 학습률 스케줄링 하기



In [1]:
#keras 라이브러리 가져오기

# import keras -> import tensorflow.keras as keras
import tensorflow.keras as keras

In [2]:
#MNIST 패션 데이터 로딩하기

fashion_mnist=keras.datasets.fashion_mnist

In [3]:
(X_train_full, y_train_full), (X_test, y_test) =fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [4]:
class_names=["T-shirt/top",'Trouser','Pullover','Dress',"Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [5]:
# 학습용 데이터를 다시 학습용 데이터와 검증용 데이터 (valdiation set) 으로 분리
# 분리와 동시에 정규화

X_valid, X_train= X_train_full[:5000]/255.0, X_train_full[5000:]/255.0
y_valid, y_train= y_train_full[:5000], y_train_full[5000:]

X_test=X_test/255.0

In [6]:
# keras 의 Sequential API 를 활용하여 신경망 모형의 골격 만들기 
# 은닉층이 2개이며 첫번째 은닉층에 뉴런 300개, 두번째 은닉층에 뉴런 100개 인 골격 만들기
# 활성함수는 relu 사용하기 


model=keras.models.Sequential()

model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300,activation="relu"))#relu 활성화 함수

model.add(keras.layers.Dense(100,activation="relu"))
model.add(keras.layers.Dense(10,activation="softmax"))#softmax 소프트맥스 함수

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


### 1.1 power scheduling

In [8]:
# 모형 컴파일하기 (손실함수, 최적화 방법 지정하기)
### power scheduling 으로 학습률을 스케줄링하기 !!  

# Ir = Ir0 / (1 + epoch / s)


optimizer=keras.optimizers.SGD(lr=0.01, decay=1e-4) #Optimizer 란? 최적화(Optimization)은 손실 함수(Loss Function)의 결과값을 최소화하는 모델의 파라미터(가중치)를 찾는 것 / 옵티마이저 안에 잇는 인자 값을 뭐? 파워스케줄링
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [9]:
#epoch 개수를 30개로 설정하여 모형 학습하기

history=model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

Epoch 1/30
1719/1719 [==============================] - 9s 5ms/step - loss: 0.7289 - accuracy: 0.7652 - val_loss: 0.5254 - val_accuracy: 0.8246
Epoch 2/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4952 - accuracy: 0.8286 - val_loss: 0.4548 - val_accuracy: 0.8462
Epoch 3/30
1719/1719 [==============================] - 8s 4ms/step - loss: 0.4546 - accuracy: 0.8406 - val_loss: 0.4498 - val_accuracy: 0.8480
Epoch 4/30
1719/1719 [==============================] - 8s 4ms/step - loss: 0.4305 - accuracy: 0.8489 - val_loss: 0.4389 - val_accuracy: 0.8436
Epoch 5/30
1719/1719 [==============================] - 8s 4ms/step - loss: 0.4141 - accuracy: 0.8550 - val_loss: 0.4008 - val_accuracy: 0.8616
Epoch 6/30
1719/1719 [==============================] - 8s 5ms/step - loss: 0.3994 - accuracy: 0.8597 - val_loss: 0.3953 - val_accuracy: 0.8672
Epoch 7/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3900 - accuracy: 0.8641 - val_loss: 0.3849 - val_accuracy:

In [10]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3691 - accuracy: 0.8682


[0.3690577745437622, 0.8682000041007996]

### 1.2 Exponential scheduling

In [11]:
# keras 의 Sequential API 를 활용하여 신경망 모형의 골격 만들기 
# 은닉층이 2개이며 첫번째 은닉층에 뉴런 300개, 두번째 은닉층에 뉴런 100개 인 골격 만들기
# 활성함수는 relu 사용하기 


model=keras.models.Sequential()

model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300,activation="relu"))

model.add(keras.layers.Dense(100,activation="relu"))
model.add(keras.layers.Dense(10,activation="softmax"))

In [12]:
# 모형 컴파일하기 (손실함수, 최적화 방법 지정하기)

model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])


In [13]:
#callback 으로 scheduler 지정하기

def exponential_decay_fn(epoch, lr):
  return lr*0.1**(epoch/20)
#안된다면 return lr*0.1**(1/20)

lr_scheduler=keras.callbacks.LearningRateScheduler(exponential_decay_fn)


In [14]:
#모형 학습하기

history=model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid), callbacks=[lr_scheduler] )

Epoch 1/30
1719/1719 [==============================] - 8s 4ms/step - loss: 0.7157 - accuracy: 0.7684 - val_loss: 0.5193 - val_accuracy: 0.8188 - lr: 0.0100
Epoch 2/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4872 - accuracy: 0.8308 - val_loss: 0.4800 - val_accuracy: 0.8318 - lr: 0.0089
Epoch 3/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4425 - accuracy: 0.8463 - val_loss: 0.4281 - val_accuracy: 0.8544 - lr: 0.0071
Epoch 4/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4177 - accuracy: 0.8541 - val_loss: 0.4095 - val_accuracy: 0.8582 - lr: 0.0050
Epoch 5/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4019 - accuracy: 0.8605 - val_loss: 0.4081 - val_accuracy: 0.8596 - lr: 0.0032
Epoch 6/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3927 - accuracy: 0.8626 - val_loss: 0.3992 - val_accuracy: 0.8622 - lr: 0.0018
Epoch 7/30
1719/1719 [==============================] - 7s

In [15]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.4215 - accuracy: 0.8525


[0.4214891493320465, 0.8525000214576721]

### 1.3 Performance scheduling

In [16]:
# keras 의 Sequential API 를 활용하여 신경망 모형의 골격 만들기 
# 은닉층이 2개이며 첫번째 은닉층에 뉴런 300개, 두번째 은닉층에 뉴런 100개 인 골격 만들기
# 활성함수는 relu 사용하기 


model=keras.models.Sequential()

model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300,activation="relu"))

model.add(keras.layers.Dense(100,activation="relu"))
model.add(keras.layers.Dense(10,activation="softmax"))

In [17]:
# 모형 컴파일하기 (손실함수, 최적화 방법 지정하기)

model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])


In [18]:
#callback 으로 scheduler 지정하기

lr_scheduler=keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)

In [ ]:
history=model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid), callbacks=[lr_scheduler] )

Epoch 1/30
1719/1719 [==============================] - 8s 4ms/step - loss: 0.7181 - accuracy: 0.7620 - val_loss: 0.5034 - val_accuracy: 0.8324 - lr: 0.0100
Epoch 2/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4890 - accuracy: 0.8288 - val_loss: 0.4652 - val_accuracy: 0.8336 - lr: 0.0100
Epoch 3/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4431 - accuracy: 0.8446 - val_loss: 0.4300 - val_accuracy: 0.8522 - lr: 0.0100
Epoch 4/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4165 - accuracy: 0.8549 - val_loss: 0.4015 - val_accuracy: 0.8618 - lr: 0.0100
Epoch 5/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3974 - accuracy: 0.8599 - val_loss: 0.3920 - val_accuracy: 0.8640 - lr: 0.0100
Epoch 6/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3805 - accuracy: 0.8660 - val_loss: 0.3754 - val_accuracy: 0.8714 - lr: 0.0100
Epoch 7/30
1719/1719 [==============================] - 7s

In [ ]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3316 - accuracy: 0.8798


[0.33156487345695496, 0.879800021648407]

## 2. Early-stopping 수행하기

### 2.1 ModelCheckpoint callback 을 이용하는 방법

In [ ]:
# keras 의 Sequential API 를 활용하여 신경망 모형의 골격 만들기 
# 은닉층이 2개이며 첫번째 은닉층에 뉴런 300개, 두번째 은닉층에 뉴런 100개 인 골격 만들기
# 활성함수는 relu 사용하기 


model=keras.models.Sequential()

model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300,activation="relu"))

model.add(keras.layers.Dense(100,activation="relu"))
model.add(keras.layers.Dense(10,activation="softmax"))

In [ ]:
# 모형 컴파일하기 (손실함수, 최적화 방법 지정하기)

model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])


In [ ]:
#callback 만들기

checkpoint_cb=keras.callbacks.ModelCheckpoint("my_keras_model.h5", monitor='val_accuracy', save_best_only=True)#ModelCheckpoint 학습 중간중간에 컴퓨터가 꺼지면 실행된게 다 없어지지 않도록 저장해두는 것 체크포인트!

In [ ]:
#모형 학습하기

history=model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid), callbacks=[checkpoint_cb])

Epoch 1/30
1719/1719 [==============================] - 4s 2ms/step - loss: 0.7145 - accuracy: 0.7661 - val_loss: 0.4908 - val_accuracy: 0.8348
Epoch 2/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4876 - accuracy: 0.8296 - val_loss: 0.4409 - val_accuracy: 0.8498
Epoch 3/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4438 - accuracy: 0.8441 - val_loss: 0.4427 - val_accuracy: 0.8486
Epoch 4/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4152 - accuracy: 0.8534 - val_loss: 0.4707 - val_accuracy: 0.8242
Epoch 5/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3954 - accuracy: 0.8601 - val_loss: 0.3919 - val_accuracy: 0.8628
Epoch 6/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3789 - accuracy: 0.8649 - val_loss: 0.3705 - val_accuracy: 0.8686
Epoch 7/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3657 - accuracy: 0.8703 - val_loss: 0.3694 - val_accuracy:

In [ ]:
model.evaluate(X_test,y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.3233 - accuracy: 0.8881


[0.3232688903808594, 0.8881000280380249]

In [ ]:
model2=keras.models.load_model("my_keras_model.h5")

In [ ]:
model2.evaluate(X_test, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.3270 - accuracy: 0.8850


[0.32698291540145874, 0.8849999904632568]

### 2.2 EarlyStopping callback 을 이용하는 방법

In [ ]:
# keras 의 Sequential API 를 활용하여 신경망 모형의 골격 만들기 
# 은닉층이 2개이며 첫번째 은닉층에 뉴런 300개, 두번째 은닉층에 뉴런 100개 인 골격 만들기
# 활성함수는 relu 사용하기 


model=keras.models.Sequential()

model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300,activation="relu"))

model.add(keras.layers.Dense(100,activation="relu"))
model.add(keras.layers.Dense(10,activation="softmax"))

In [ ]:
# 모형 컴파일하기 (손실함수, 최적화 방법 지정하기)

model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])


In [ ]:
#callback 만들기

early_stopping_cb=keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# option : monitor= 

In [ ]:
#모형 학습하기

history=model.fit(X_train, y_train, epochs=50, validation_data=(X_valid, y_valid), callbacks=[checkpoint_cb, early_stopping_cb])


Epoch 1/50
1719/1719 [==============================] - 4s 2ms/step - loss: 0.7142 - accuracy: 0.7646 - val_loss: 0.5133 - val_accuracy: 0.8280
Epoch 2/50
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4885 - accuracy: 0.8305 - val_loss: 0.4498 - val_accuracy: 0.8484
Epoch 3/50
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4431 - accuracy: 0.8438 - val_loss: 0.4387 - val_accuracy: 0.8498
Epoch 4/50
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4158 - accuracy: 0.8546 - val_loss: 0.3973 - val_accuracy: 0.8630
Epoch 5/50
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3945 - accuracy: 0.8623 - val_loss: 0.3968 - val_accuracy: 0.8636
Epoch 6/50
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3776 - accuracy: 0.8680 - val_loss: 0.3790 - val_accuracy: 0.8682
Epoch 7/50
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3646 - accuracy: 0.8720 - val_loss: 0.3696 - val_accuracy:

In [ ]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.3520 - accuracy: 0.8761


[0.3519742786884308, 0.8761000037193298]

## 3. 초모수를 자동으로 튜닝하기

In [ ]:
#데이터셋 로딩, 데이터셋 분리, 데이터 정규화에 필요한 패키지 로딩하기

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
#데이터셋 로딩하기

housing=fetch_california_housing()

In [ ]:
# 데이터를 학습용 데이터와 테스트용 데이터로 나누기
# 학습용 데이터를 다시 학습용 데이터와 검증용 데이터로 나누기

X_train_full, X_test, y_train_full, y_test= train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [ ]:
# 데이터 정규화 하기

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_valid=scaler.transform(X_valid)
X_test=scaler.transform(X_test)

In [ ]:
# keras 의 Sequential API 를 활용하여 신경망 모형의 골격 만들기 
# 은닉층이 1개이며 뉴런이 30개 인 골격 만들기
# 활성함수는 relu 사용하기

model=keras.models.Sequential([
  keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
  keras.layers.Dense(1)
])

In [ ]:
#모형 컴파일하기(손실함수, 최적화 방법 지정하기) 

model.compile(loss="mean_squared_error", optimizer="sgd")


In [ ]:
#epoch 개수를 20개로 설정하여 모형 학습하기

history=model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 1.2989 - val_loss: 0.6539
Epoch 2/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5384 - val_loss: 0.4603
Epoch 3/20
363/363 [==============================] - 1s 1ms/step - loss: 0.4507 - val_loss: 0.4370
Epoch 4/20
363/363 [==============================] - 0s 990us/step - loss: 0.4418 - val_loss: 0.4189
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4160 - val_loss: 0.4077
Epoch 6/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4078 - val_loss: 0.3990
Epoch 7/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3990 - val_loss: 0.4014
Epoch 8/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3912 - val_loss: 0.3886
Epoch 9/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3855 - val_loss: 0.3838
Epoch 10/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3824 - val_loss: 0.39

In [ ]:
#학습 결과 확인하기

mse_test=model.evaluate(X_test,y_test)

162/162 [==============================] - 0s 825us/step - loss: 0.3764


### 3.1 위의 전 과정을 수행하는 KerasRegressor 의 생성

In [ ]:
# 모형의 골격 및 컴파일 과정을 하나의 함수로 만들기

def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
  model=keras.models.Sequential()
  model.add(keras.layers.InputLayer(input_shape=input_shape))
  for layer in range(n_hidden):
    model.add(keras.layers.Dense(n_neurons, activation="relu"))
  model.add(keras.layers.Dense(1))
  optimizer=keras.optimizers.SGD(lr=learning_rate)
  model.compile(loss="mse",optimizer=optimizer)
  return model

In [ ]:
#KerasRegressor 생성하기

keras_reg=keras.wrappers.scikit_learn.KerasRegressor(build_model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#KerasRegressor 를 일반 sklearn 의 regressor 처럼 사용하여 모형 학습하기

keras_reg.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


363/363 [==============================] - 1s 1ms/step - loss: 1.5318 - val_loss: 0.6544
Epoch 2/100
363/363 [==============================] - 0s 1ms/step - loss: 0.6245 - val_loss: 0.5655
Epoch 3/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5502 - val_loss: 0.5367
Epoch 4/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5133 - val_loss: 0.5029
Epoch 5/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4866 - val_loss: 0.5014
Epoch 6/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4700 - val_loss: 0.4633
Epoch 7/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4611 - val_loss: 0.4480
Epoch 8/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4424 - val_loss: 0.4519
Epoch 9/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4317 - val_loss: 0.4265
Epoch 10/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4258 - val_loss: 0.4330
E

In [ ]:
mse_test=keras_reg.score(X_test, y_test)

162/162 [==============================] - 0s 770us/step - loss: 0.3801


In [ ]:
X_new=X_test[:3]
y_pred=keras_reg.predict(X_new)

### 3.2 KerasRegressor 와 RandomizedSearchCV 를 이용하여 초모수 선택을 자동화하기

In [ ]:
#필요 패키지 불러오기

from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

import numpy as np

In [ ]:
#초모수 후보 (KerasRegressor 의 input) 지정하기

param_distribs={
    "n_hidden":(0,1,2,3),
    "n_neurons": (10,20),
    "learning_rate": (3e-4, 3e-2)
}

In [ ]:
rnd_search_cv=RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(X_train,y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


242/242 [==============================] - 1s 2ms/step - loss: 2.8385 - val_loss: 1.2663
Epoch 2/100
242/242 [==============================] - 0s 2ms/step - loss: 1.3298 - val_loss: 1.2676
Epoch 3/100
242/242 [==============================] - 0s 2ms/step - loss: 1.3301 - val_loss: 1.2660
Epoch 4/100
242/242 [==============================] - 0s 1ms/step - loss: 1.3302 - val_loss: 1.2657
Epoch 5/100
242/242 [==============================] - 0s 1ms/step - loss: 1.3303 - val_loss: 1.2653
Epoch 6/100
242/242 [==============================] - 0s 1ms/step - loss: 1.3299 - val_loss: 1.2679
Epoch 7/100
242/242 [==============================] - 0s 1ms/step - loss: 1.3303 - val_loss: 1.2659
Epoch 8/100
242/242 [==============================] - 0s 1ms/step - loss: 1.3301 - val_loss: 1.2686
Epoch 9/100
242/242 [==============================] - 0s 1ms/step - loss: 1.3302 - val_loss: 1.2656
Epoch 10/100
242/242 [==============================] - 0s 2ms/step - loss: 1.3299 - val_loss: 1.2681
E

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


242/242 [==============================] - 1s 2ms/step - loss: 0.6747 - val_loss: 0.6600
Epoch 2/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4832 - val_loss: 0.4222
Epoch 3/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4252 - val_loss: 0.4627
Epoch 4/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4579 - val_loss: 0.4323
Epoch 5/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4167 - val_loss: 0.3941
Epoch 6/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4340 - val_loss: 0.4006
Epoch 7/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3869 - val_loss: 0.4167
Epoch 8/100
242/242 [==============================] - 0s 1ms/step - loss: 0.3935 - val_loss: 0.4092
Epoch 9/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3791 - val_loss: 0.3739
Epoch 10/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3722 - val_loss: 0.3913
E

...

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


242/242 [==============================] - 1s 3ms/step - loss: 5.0662 - val_loss: 3.5115
Epoch 2/100
242/242 [==============================] - 0s 1ms/step - loss: 2.6935 - val_loss: 2.0116
Epoch 3/100
242/242 [==============================] - 0s 1ms/step - loss: 1.6069 - val_loss: 1.3098
Epoch 4/100
242/242 [==============================] - 0s 2ms/step - loss: 1.1150 - val_loss: 0.9896
Epoch 5/100
242/242 [==============================] - 0s 1ms/step - loss: 0.9011 - val_loss: 0.8438
Epoch 6/100
242/242 [==============================] - 0s 1ms/step - loss: 0.8072 - val_loss: 0.7743
Epoch 7/100
242/242 [==============================] - 0s 2ms/step - loss: 0.7623 - val_loss: 0.7374
Epoch 8/100
242/242 [==============================] - 0s 1ms/step - loss: 0.7376 - val_loss: 0.7154
Epoch 9/100
242/242 [==============================] - 0s 2ms/step - loss: 0.7212 - val_loss: 0.6999
Epoch 10/100
242/242 [==============================] - 0s 2ms/step - loss: 0.7090 - val_loss: 0.6884
E

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [-6.73537234e-01 -3.21659187e-01 -5.67467690e-01 -3.24240880e+24
 -3.36164355e-01             nan -4.35703496e-01 -3.71636407e+24
 -5.50220966e-01 -5.01210282e-01]
  category=UserWarning,


363/363 [==============================] - 1s 2ms/step - loss: 0.6151 - val_loss: 0.4493
Epoch 2/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4066 - val_loss: 0.4668
Epoch 3/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3936 - val_loss: 0.4698
Epoch 4/100
363/363 [==============================] - 1s 1ms/step - loss: 0.4835 - val_loss: 0.4010
Epoch 5/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5895 - val_loss: 0.4047
Epoch 6/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3829 - val_loss: 0.3589
Epoch 7/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3552 - val_loss: 0.3579
Epoch 8/100
363/363 [==============================] - 1s 1ms/step - loss: 0.3429 - val_loss: 0.3346
Epoch 9/100
363/363 [==============================] - 1s 1ms/step - loss: 0.3333 - val_loss: 0.3609
Epoch 10/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3272 - val_loss: 0.3218
E

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7efd3f4242d0>,
                   param_distributions={'learning_rate': (0.0003, 0.03),
                                        'n_hidden': (0, 1, 2, 3),
                                        'n_neurons': (10, 20)})

In [ ]:
#선택된 초모수 출력하기

rnd_search_cv.best_params_

{'n_neurons': 20, 'n_hidden': 3, 'learning_rate': 0.03}

In [ ]:
#선택된 초모수에서의 score 출력하기

rnd_search_cv.best_score_

-0.32165918747584027

In [ ]:
#선택된 초모수에 해당하는 모형 저장하기 

model=rnd_search_cv.best_estimator_.model